In [0]:
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from itertools import chain

class preprocess():
    def __init__(self , data , target , weight , labels = None):
        self.data = data
        self.weight = weight
        self.cntlimit = 0.02
        self.evntlimit = 0.02
        self.target = target
        self.X = None
        self.Y =  target
        self.n = 15
        self.df1 = data
        self.iv = None
        self.iv_df = None
        self.new_iv = None
        self.fv= None 
        self.fv1 = None
        self.data = data
        self.labels = target
        self.new_collinear = None        
        self.base_features = list(data.columns)
        self.one_hot_features = None       
        self.record_missing = None
        self.record_single_unique = None
        self.record_collinear = None
        self.record_zero_importance = None
        self.record_low_importance = None
        self.missing_stats = None
        self.unique_stats = None
        self.corr_matrix = None
        self.feature_importances = None
        self.ops = {}
        self.one_hot_correlated = False
        
    def mono_try(self , Y , X , weight ,n , cntlimit , evntlimit): 
        k = 0
        l = 0
        r = 0
        n1 = 0
        flag = 0
        self.X = X 
        self.Y = Y 
        self.weight = weight 
        self.cntlimit = cntlimit 
        self.evntlimit = evntlimit 
        df1 = pd.DataFrame({"X": X, "Y": Y , "weight":weight})
        df1['Y1'] = df1['Y'] * df1['weight']
        justmiss = df1[['X','Y','Y1','weight']][df1.X.isnull()]
        notmiss = df1[['X','Y','Y1','weight']][df1.X.notnull()]
        self.n = n
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        if len(justmiss.index) > 0:
            d4["MAX_VALUE"] = np.nan
            d4["COUNT"] = justmiss.sum().weight
            d4["EVENT"] = justmiss.sum().Y1
            d4["NONEVENT"] = justmiss.sum().weight - justmiss.sum().Y1
        while(np.abs(r) < 1.0 or (np.abs(k) < cntlimit or np.abs(l) < evntlimit)):
            if flag > n : 
                break
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y1,"weight":weight, "Bucket": pd.qcut(notmiss.X, n , duplicates='drop')})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n-1
            if len(d2) == 1:
                n = 5        
                bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
                if len(np.unique(bins)) == 2:
                    bins = np.insert(bins, 0, 1)
                    bins[1] = bins[1]-(bins[1]/2)
                d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y1,"weight":weight, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
                d2 = d1.groupby('Bucket', as_index=True)
                d3 = pd.DataFrame({},index=[])
                d3["MIN_VALUE"] = d2.min().X
                d3["MAX_VALUE"] = d2.max().X
                d3["COUNT"] = d2.sum().weight
                d3["EVENT"] = d2.sum().Y
                d3["NONEVENT"] = d2.sum().weight - d2.sum().Y
                d3=d3.reset_index(drop=True)
                if len(d4.index) > 0:
                    d3 = d3.append(d4,ignore_index=True)
                d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
                d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
                d3["DIST_COUNT"] = d3.COUNT/d3.sum().COUNT
                d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
                d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
                d3["WOE"] = np.log(d3.DIST_NON_EVENT/d3.DIST_EVENT)
                d3["IV_indiv"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*d3['WOE']
                d3["VAR_NAME"] = "VAR"
                d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT','DIST_COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT','NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE','IV_indiv']]                       
                d3 = d3.replace([np.inf, -np.inf], 0)
                flag +=1 
                d3['FORCE'] = 1
            else:
                d3 = pd.DataFrame({},index=[])
                d3["MIN_VALUE"] = d2.min().X
                d3["MAX_VALUE"] = d2.max().X
                d3["COUNT"] = d2.sum().weight
                d3["EVENT"] = d2.sum().Y
                d3["NONEVENT"] = d2.sum().weight - d2.sum().Y
                d3=d3.reset_index(drop=True)
                if(len(d4.index > 0)):
                    d3 = d3.append(d4,ignore_index=True)
                d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
                d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
                d3["DIST_COUNT"] = d3.COUNT/d3.sum().COUNT
                d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
                d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
                d3["WOE"] = np.log(d3.DIST_NON_EVENT/d3.DIST_EVENT)
                d3["IV_indiv"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*d3['WOE']
                d3["VAR_NAME"] = "VAR"                     
                d3 = d3.replace([np.inf, -np.inf], 0)
                l = d3['DIST_EVENT'].min()
                k = d3['DIST_COUNT'].min()
                d3['FORCE'] = 0
        d3['IV'] = (d3['DIST_NON_EVENT'] - d3['DIST_EVENT'])*(np.log(d3.DIST_NON_EVENT/d3.DIST_EVENT))
        d3 = d3.replace([np.inf, -np.inf], 0)
        d3['IV'] = (d3.IV.sum())
        return (d3)
    
    def char_bin_wt(self , Y, X , weight):
        self.Y = Y
        self.X = X
        self.weight = weight
        df1 = pd.DataFrame({"X": X, "Y": Y , 'weight':weight})
        df1['Y1'] = df1['Y'] * df1['weight']
        justmiss = df1[['X','Y1','weight']][df1.X.isnull()]
        notmiss = df1[['X','Y1','weight']][df1.X.notnull()]    
        df2 = notmiss.groupby('X',as_index=True)

        d3 = pd.DataFrame({},index=[])
        d3["COUNT"] = df2.sum().weight
        d3["MIN_VALUE"] = df2.sum().Y1.index
        d3["MAX_VALUE"] = d3["MIN_VALUE"]
        d3["EVENT"] = df2.sum().Y1
        d3["NONEVENT"] = df2.sum().weight - df2.sum().Y1

        if len(justmiss.index) > 0:
            d4 = pd.DataFrame({'MIN_VALUE':-np.inf},index=[0])
            d4["MAX_VALUE"] = np.nan
            d4["COUNT"] = justmiss.sum().weight
            d4["EVENT"] = justmiss.sum().Y1
            d4["NONEVENT"] = justmiss.sum().weight - justmiss.sum().Y1
            d3 = d3.append(d4,ignore_index=True)

        d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
        d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
        d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
        d3["DIST_COUNT"] = d3.COUNT/d3.sum().COUNT
        d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
        d3["WOE"] = np.log(d3.DIST_NON_EVENT/d3.DIST_EVENT)
        d3["IV_indiv"] = (d3.DIST_NON_EVENT-d3.DIST_EVENT)*np.log(d3.DIST_NON_EVENT/d3.DIST_EVENT)
        d3["IV"] = (d3.DIST_NON_EVENT-d3.DIST_EVENT)*np.log(d3.DIST_NON_EVENT/d3.DIST_EVENT)
        d3["VAR_NAME"] = "VAR"
        d3 = d3.replace([np.inf, -np.inf], 0)
        d3['FORCE'] = 2
        d3.IV = d3.IV.sum()
        d3 = d3.reset_index(drop=True)
        d3 = d3.sort_values(by = 'WOE')
        if len(df1['X'].value_counts()) < 3 :
            return d3
        else :
            d3['diff'] = d3['WOE'].sub(d3['WOE'].shift())
            d3['diff'] = d3['diff'].fillna(0.0)
            d3['diff2'] = d3['diff'].cumsum() * 10
            d3['bin'] = (d3['diff2']).astype('int')
            d3['MIN_VALUE'] = d3['MIN_VALUE'].astype('str')
            grouped_df = d3.groupby("bin")['MIN_VALUE'].apply(lambda x: "%s" % ','.join(x))
            d_check = d3[['COUNT','EVENT','NONEVENT','bin']].merge(grouped_df.to_frame().reset_index() , on = 'bin')
            del d3
            d_check = d_check.drop(['bin'],axis = 1)    
            df2 = d_check.groupby('MIN_VALUE', as_index = True)
            d3 = pd.DataFrame({},index=[])
            d3["COUNT"] = df2.sum().COUNT
            d3["MIN_VALUE"] = df2.sum().COUNT.index
            d3["MAX_VALUE"] = d3["MIN_VALUE"]
            d3["EVENT"] = df2.sum().EVENT
            d3["NONEVENT"] = df2.sum().NONEVENT
            d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
            d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
            d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
            d3["DIST_COUNT"] = d3.COUNT/d3.sum().COUNT
            d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
            d3["WOE"] = np.log(d3.DIST_NON_EVENT/d3.DIST_EVENT)
            d3["IV_indiv"] = (d3.DIST_NON_EVENT-d3.DIST_EVENT)*np.log(d3.DIST_NON_EVENT/d3.DIST_EVENT)
            d3["IV"] = (d3.DIST_NON_EVENT-d3.DIST_EVENT)*np.log(d3.DIST_NON_EVENT/d3.DIST_EVENT)
            d3["VAR_NAME"] = "VAR"
            d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT','DIST_COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT','NON_EVENT_RATE','DIST_EVENT','DIST_NON_EVENT','WOE', 'IV_indiv' , 'IV']]      
            d3 = d3.replace([np.inf, -np.inf], 0)
            d3['FORCE'] = 2
            d3.IV = d3.IV.sum()
            d3 = d3.reset_index(drop=True)
            return d3
        
    def binning(self, evntlimit = 0.03 , cntlimit = 0.03):
        self.n = 15
        count = 0
        iv_df = pd.DataFrame({} , index = [])
        for i in list(self.data.columns):
            if ((np.issubdtype(self.data[i], np.number)) and (len(Series.unique(self.data[i])) > 2) and (len(self.data[i].value_counts()) > 2)):
                conv = self.mono_try(self.target, self.data[i] , self.weight , 15 , cntlimit , evntlimit)
                conv["VAR_NAME"] = i
                count = count + 1
                print(i + ' is completed...')
            else:
                conv = self.char_bin_wt(self.target, self.data[i] , self.weight)
                conv["VAR_NAME"] = i            
                count = count + 1
                print(i + " is completed...")
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
        iv = iv_df[['VAR_NAME','IV']].drop_duplicates().reset_index(drop=True)
        iv_df = iv_df[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT','DIST_COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT','NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE','FORCE','IV_indiv','IV']]
        self.iv = iv
        self.iv_df =  iv_df
        return(iv_df  , iv)
    
    def tidy_split(self , column, sep='|', keep=False):    
        df = self.fv1
        indexes = list()
        new_values = list()
        for i, presplit in enumerate(df[column].astype(str)):
            values = presplit.split(sep)
            if keep and len(values) > 1:
                indexes.append(i)
                new_values.append(presplit)
            for value in values:
                indexes.append(i)
                new_values.append(value)
        new_df = df.iloc[indexes, :].copy()
        new_df[column] = new_values
        return new_df  
    
    def preprocess(self):
        final_iv = self.iv_df
        a = pd.to_numeric(final_iv.MIN_VALUE, errors='coerce')
        final_iv  = final_iv[~final_iv['WOE'].isna()]
        a = pd.to_numeric(final_iv.MIN_VALUE, errors='coerce')
        idx = a.isna()
        l1 = list(final_iv[idx]['VAR_NAME'].unique())
        fv1 = final_iv[final_iv['VAR_NAME'].isin(l1)]
        fv2 = final_iv[~final_iv['VAR_NAME'].isin(l1)]
        self.fv1 = fv1 
        finaliv2 = self.tidy_split('MIN_VALUE' , ',')
        finaliv2['MAX_VALUE']  =  finaliv2['MIN_VALUE']
        fv = fv2.append(finaliv2 , ignore_index = True)
        self.fv = fv
        
    def transform(self):
        df = self.df1
        self.preprocess()
        final_iv = self.fv
        list_col = list(final_iv.VAR_NAME.unique())
        df_t = df[list_col]
        for i in list_col:
            if df_t[i].dtypes == 'object':
                booleanDictionary = {True: 'TRUE', False: 'FALSE'}
                df_t[i] = df_t[i].replace(booleanDictionary)
                final_i1 = final_iv.loc[final_iv.VAR_NAME == i].reset_index()
                final_i1['MAX_VALUE'] = final_i1['MAX_VALUE'].replace(booleanDictionary)
                cut1 = np.hstack([final_i1['MAX_VALUE']])
                lab = final_i1['WOE'].round(4)
                df_t[i] = df_t[i].replace(cut1 , lab)
                if(df_t[i].isnull().sum() > 0):
                    k = (final_i1[final_i1['MAX_VALUE'].isnull()]['WOE']).values
                    df_t[i] = dtst.fillna(k[0])
                else:
                    df_t[i] = df_t[i]
            else :
                df_t[i] = df_t[i].astype('float64')
                final_i1 = final_iv.loc[final_iv.VAR_NAME == i].reset_index()
                final_i1.MAX_VALUE = final_i1.MAX_VALUE.fillna(np.nan).astype(float)
                final_i1 = final_i1.sort_values(by = 'MAX_VALUE' , ascending =True).reset_index(drop = True)
                if(len(final_iv) == 1) : 
                    cutoff = np.hstack([np.array(final_i1['MIN_VALUE'][0]), final_i1['MAX_VALUE'].values + 0.0001])
                    cutoff = cutoff.astype('float64')
                else :     
                    if(final_i1['MIN_VALUE'][0] == final_i1['MAX_VALUE'][1]):
                        cutoff = np.hstack([final_i1['MAX_VALUE'].values])
                        cutoff = cutoff.astype('float64')
                    else:
                        cutoff = np.hstack([np.array(final_i1['MIN_VALUE'][0]), final_i1['MAX_VALUE'].values + 0.0001])
                        cutoff = cutoff.astype('float64')
                print(cutoff)
                labels = final_i1['WOE']
                print(labels)
                dtst = pd.cut(df_t[i], bins=cutoff, labels=labels, include_lowest=True , duplicates='drop')
                if(df_t[i].isnull().sum()>0):
                    k = (final_i1[final_i1['MAX_VALUE'].isnull()]['WOE']).values
                    print(k)
                    df_t[i] = dtst.fillna(k[0])
                else:
                    df_t[i] = dtst
        return df_t.round(4) 

    def identify_missing(self, missing_threshold):        
        self.missing_threshold = missing_threshold

        missing_series = self.data.isnull().sum() / self.data.shape[0]
        self.missing_stats = pd.DataFrame(missing_series).rename(columns = {'index': 'feature', 0: 'missing_fraction'})

        self.missing_stats = self.missing_stats.sort_values('missing_fraction', ascending = False)

        record_missing = pd.DataFrame(missing_series[missing_series > missing_threshold]).reset_index().rename(columns = 
                                                                                                               {'index': 'feature', 
                                                                                                                0: 'missing_fraction'})

        to_drop = list(record_missing['feature'])

        self.record_missing = record_missing
        self.ops['missing'] = to_drop
        
        print('%d features with greater than %0.2f missing values.\n' % (len(self.ops['missing']), self.missing_threshold))
        
    def identify_single_unique(self):
        unique_counts = self.data.nunique()
        self.unique_stats = pd.DataFrame(unique_counts).rename(columns = {'index': 'feature', 0: 'nunique'})
        self.unique_stats = self.unique_stats.sort_values('nunique', ascending = True)
        
        record_single_unique = pd.DataFrame(unique_counts[unique_counts == 1]).reset_index().rename(columns = {'index': 'feature', 
                                                                                                                0: 'nunique'})

        to_drop = list(record_single_unique['feature'])
    
        self.record_single_unique = record_single_unique
        self.ops['single_unique'] = to_drop
        
        print('%d features with a single unique value.\n' % len(self.ops['single_unique']))
    
    def identify_collinear(self, correlation_threshold, one_hot=False):
        self.correlation_threshold = correlation_threshold
        self.one_hot_correlated = one_hot
        
        if one_hot:
            
            features = pd.get_dummies(self.data)
            self.one_hot_features = [column for column in features.columns if column not in self.base_features]

            self.data_all = pd.concat([features[self.one_hot_features], self.data], axis = 1)
            
            corr_matrix = pd.get_dummies(features).corr()

        else:
            corr_matrix = self.data.corr()
        
        self.corr_matrix = corr_matrix
    
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
        
        to_drop = [column for column in upper.columns if any(upper[column].abs() > correlation_threshold)]

        record_collinear = pd.DataFrame(columns = ['drop_feature', 'corr_feature', 'corr_value'])

        for column in to_drop:

            corr_features = list(upper.index[upper[column].abs() > correlation_threshold])

            corr_values = list(upper[column][upper[column].abs() > correlation_threshold])
            drop_features = [column for _ in range(len(corr_features))]    

            temp_df = pd.DataFrame.from_dict({'drop_feature': drop_features,
                                             'corr_feature': corr_features,
                                             'corr_value': corr_values})

            record_collinear = record_collinear.append(temp_df, ignore_index = True)

        self.record_collinear = record_collinear
        self.ops['collinear'] = to_drop
        
        print('%d features with a correlation magnitude greater than %0.2f.\n' % (len(self.ops['collinear']), self.correlation_threshold))
        
    def identify_all(self, selection_params):
        self.identify_missing(selection_params['missing_threshold'])
        self.identify_single_unique()
        self.identify_collinear(selection_params['correlation_threshold'])
        self.binning()
        self.drop_ivcor()  
        self.all_identified = set(list(chain(*list(self.ops.values()))))
        self.n_identified = len(self.all_identified)
        
        
    def check_removal(self, keep_one_hot=True):        
        self.all_identified = set(list(chain(*list(self.ops.values()))))
        print('Total of %d features identified for removal' % len(self.all_identified))
        
        if not keep_one_hot:
            if self.one_hot_features is None:
                print('Data has not been one-hot encoded')
            else:
                one_hot_to_remove = [x for x in self.one_hot_features if x not in self.all_identified]
                print('%d additional one-hot features can be removed' % len(one_hot_to_remove))
        
        return list(self.all_identified)
        
    
    def remove(self, methods, keep_one_hot = True):
        features_to_drop = []
      
        if methods == 'all':
            
            data = self.data_all
                                          
            print('{} methods have been run\n'.format(list(self.ops.keys())))
            
            features_to_drop = set(list(chain(*list(self.ops.values()))))
            
        else:
            if 'zero_importance' in methods or 'low_importance' in methods or self.one_hot_correlated:
                data = self.data_all
                
            else:
                data = self.data
                
            for method in methods:
                
                if method not in self.ops.keys():
                    raise NotImplementedError('%s method has not been run' % method)
                    
                else:
                    features_to_drop.append(self.ops[method])
        
            features_to_drop = set(list(chain(*features_to_drop)))
            
        features_to_drop = list(features_to_drop)
            
        if not keep_one_hot:
            
            if self.one_hot_features is None:
                print('Data has not been one-hot encoded')
            else:
                             
                features_to_drop = list(set(features_to_drop) | set(self.one_hot_features))
       
        data = data.drop(columns = features_to_drop)
        self.removed_features = features_to_drop
        
        if not keep_one_hot:
            print('Removed %d features including one-hot features.' % len(features_to_drop))
        else:
            print('Removed %d features.' % len(features_to_drop))
        
        return data
    
    def plot_missing(self):
        """Histogram of missing fraction in each feature"""
        if self.record_missing is None:
            raise NotImplementedError("Missing values have not been calculated. Run `identify_missing`")
        
        self.reset_plot()
        
        plt.style.use('seaborn-white')
        plt.figure(figsize = (7, 5))
        plt.hist(self.missing_stats['missing_fraction'], bins = np.linspace(0, 1, 11), edgecolor = 'k', color = 'red', linewidth = 1.5)
        plt.xticks(np.linspace(0, 1, 11));
        plt.xlabel('Missing Fraction', size = 14); plt.ylabel('Count of Features', size = 14); 
        plt.title("Fraction of Missing Values Histogram", size = 16);
        
    
    def plot_unique(self):
        if self.record_single_unique is None:
            raise NotImplementedError('Unique values have not been calculated. Run `identify_single_unique`')
        
        self.reset_plot()

        self.unique_stats.plot.hist(edgecolor = 'k', figsize = (7, 5))
        plt.ylabel('Frequency', size = 14); plt.xlabel('Unique Values', size = 14); 
        plt.title('Number of Unique Values Histogram', size = 16);
        
    
    def plot_collinear(self, plot_all = False):
        if self.record_collinear is None:
            raise NotImplementedError('Collinear features have not been idenfitied. Run `identify_collinear`.')
        
        if plot_all:
            corr_matrix_plot = self.corr_matrix
            title = 'All Correlations'
        
        else:
            
            corr_matrix_plot = self.corr_matrix.loc[list(set(self.record_collinear['corr_feature'])),                            list(set(self.record_collinear['drop_feature']))]
            title = "Correlations Above Threshold"

       
        f, ax = plt.subplots(figsize=(10, 8))
        
        cmap = sns.diverging_palette(220, 10, as_cmap=True)

        sns.heatmap(corr_matrix_plot, cmap=cmap, center=0,
                    linewidths=.25, cbar_kws={"shrink": 0.6})

        ax.set_yticks([x + 0.5 for x in list(range(corr_matrix_plot.shape[0]))])
        ax.set_yticklabels(list(corr_matrix_plot.index), size = int(160 / corr_matrix_plot.shape[0]));

        ax.set_xticks([x + 0.5 for x in list(range(corr_matrix_plot.shape[1]))])
        ax.set_xticklabels(list(corr_matrix_plot.columns), size = int(160 / corr_matrix_plot.shape[1]));
        plt.title(title, size = 14)
        
    def reset_plot(self):
        plt.rcParams = plt.rcParamsDefault
        
    def drop_ivcor(self):
        self.new_collinear = self.record_collinear.merge(self.iv.rename(columns={'VAR_NAME':'drop_feature','IV':'drop_iv'}) , on = 'drop_feature').merge(self.iv.rename(columns={'VAR_NAME':'corr_feature','IV':'corr_iv'}))
        self.new_collinear['drop_feature1'] = np.where(self.new_collinear['drop_iv'] < self.new_collinear['corr_iv'] , self.new_collinear['drop_feature'] , self.new_collinear['corr_feature'])
        to_drop = list(self.new_collinear['drop_feature1'].unique())
        self.ops['IVcollinear'] = to_drop
        return list(self.new_collinear['drop_feature1'].unique())